## Running the trained model on the test data

In [15]:
import json
import jsonlines
from transformers import pipeline, AutoTokenizer,AutoModelForSeq2SeqLM

Initializing model and tokenizer:

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('contest_train') #or any other path of yours
tokenizer = AutoTokenizer.from_pretrained('contest_train')

Opening the test data:

In [ ]:
with jsonlines.open('test.jsonl')

In [13]:
with open('test.jsonl', 'r') as f:
    data = f.readlines()

In [14]:
data

['{"translation": {"ru": "Эритроциты: отрицательно"}}\n',
 '{"translation": {"ru": "При осмотре обнаруживаются двусторонние припухлости вокруг моляров."}}\n',
 '{"translation": {"ru": "Он испытывает замедленность движений и тремор."}}\n',
 '{"translation": {"ru": "Она редко занимается спортом, и движение, похоже, дается ей с трудом."}}\n',
 '{"translation": {"ru": "Нижние конечности нормальные, без отеков."}}\n',
 '{"translation": {"ru": "У нее нет некоторых."}}\n',
 '{"translation": {"ru": "Недавняя биопсия была отрицательной на гепатоцеллюлярную карциному и отличалась только хроническим воспалением, совместимым с хроническим вирусным гепатитом."}}\n',
 '{"translation": {"ru": "Это женщина с ожирением, не имеющая острых расстройств."}}\n',
 '{"translation": {"ru": "За последние 2 года у нее был только один партнер, и последние два года она не использовала никаких методов контрацепции."}}\n',
 '{"translation": {"ru": "У него снова не было значительного ответа, хотя он продолжал иметь о

In [ ]:
data = [x.strip('\n') for x in data]

Writing the preprocessing function:

In [ ]:
def get_translation(text, lang, cur_model, cur_tokenizer):
    if lang == 'ru':
        translator = pipeline("translation_en_to_ru", model=cur_model, tokenizer=cur_tokenizer)
    else:
        translator = pipeline("translation_ru_to_en", model=cur_model, tokenizer=cur_tokenizer)
    text_length = len(text.split())
    text_split = str(text).split()
    if text_length > 100:
        sentences = sent_tokenize(text)
        result = []
        for sent in sentences:
            result.append(translator(sent)[0]['translation_text'])
        result = ' '.join(result)

    else:
        translation = translator(text)
        result = translation[0]['translation_text']    

    return result

Running the translation:

In [ ]:
preds = []

for el in data:
    translated = get_translation(el, 'en', cur_model=model, cur_tokenizer=tokenizer)
    preds.append({"translation": {"ru": el, "en": translated}})

Saving the predictions into the desired format:

In [ ]:
with open('preds.json', 'w') as f:
      json.dumps(preds)